# Donkey library tools

Let's take a look the tools provided by the [Donkey](https://github.com/wroscoe/donkey) library.

## The Donkey library

The [Donkey library](https://github.com/wroscoe/donkey) has several components.

It is first and foremost a python library installed where your other python libraries are (e.g. system python or virtualenv). After installation, you can `import` it as any normal python library:

```python
import donkeycar as dk
```

A `Vehicle` application, installed to the `~/d2` directory by default. This is where you'll find the `manage.py` script, which is used for both **driving** and **training**.

```bash
~/d2/manage.py --help
```

**Finally**, it has a CLI with tools mainly used to aid training. This is what we'll dig into in this chapter.

```bash
donkey --help
```

You'll find the complete CLI API documentation here:
- [http://docs.donkeycar.com/utility/donkey/](http://docs.donkeycar.com/utility/donkey/)

### Install the Donkey library

If you already installed the [Donkey](https://github.com/wroscoe/donkey) library, you can [skip](#Train) this step.

Otherwise, go ahead:

In [ ]:
# Make sure we're in SageMaker root
%cd ~/SageMaker

# Remove any old versions of the library
!rm -rf ~/SageMaker/donkey

# Clone the Donkey library git
!git clone https://github.com/wroscoe/donkey.git

In [ ]:
# Update Donkey dependencies

# Keras is pinned to version 2.0.8 in the Donkey requirements. Change this to allow a newer version
!sed -i -e 's/keras==2.0.8/keras>=2.1.2/g' ~/SageMaker/donkey/setup.py
!sed -i -e 's/tensorflow>=1.1/tensorflow-gpu>=1.4/g' ~/SageMaker/donkey/setup.py

# Install
!pip uninstall --yes donkeycar
!pip install ~/SageMaker/donkey

In [ ]:
# Test the CLI
!donkey --help

## Download sample data

We've created some sample data for you start working on, so that you don't have to wait for your car to be ready. Since he sample data is recorded on another car on another track, it might not be representative for you car. However, it will allow you to get started, and it will provide a good foundation for you to continue training once you get data from you own car.

If you already downloaded the sample data, you can [skip](#The-Donkey-CLI) this step.

Otherwise, go ahead and download the sample driving run, called *Tubs*, in Donkey:

In [ ]:
from sagemaker import get_execution_role

# Bucket location to get training data
sample_data_location = 's3://jayway-robocar-raw-data/samples'

In [ ]:
# Create a data directory
!mkdir -pv ~/SageMaker/data
!aws s3 cp {sample_data_location}/ore.zip ~/SageMaker/data

In [ ]:
# Unzip to data dir
!unzip -o ~/SageMaker/data/ore.zip -d ~/SageMaker/data

## The Donkey CLI

### Car related commands

Three of the commands relate to the managing and driving of a *Donkey car*:
- `findcar` - Tries to discover the IP-address of your car on the network. The search is restricted to the same subnet as the host (/24), and based on the MAC addresses of Raspberry PIs, hence it will return the IP-address of all Raspberry PIs connected to your subnet (not only Donkey cars).
- `calibrate` - Calibrate *throttle* and *steering* on a newly installed car.
- `createcar` - See [createcar](#createcar)

The `createcar` and `calibrate` commands are used in the [prepare car](../docs/PREPARE-CAR.md) track (see [README](../README.md)), and are further explained there.

The `createcar` command is also used when starting training, so we will have a closer look at it here

### createcar

Create a new *Donkey car application* to the specified *path* based on the specified *template*.

The default *Donkey car application* allows you to **drive** and **train** your car.

A template specifies what *Donkey parts* should be a part of this *Donkey car application*. Templates can be found in `donkey/donkeycar/templates`:

In [ ]:
# This command assumes you've installed the Donkey library to the project root
!ls -l ./donkey/donkeycar/templates

In [ ]:
# Create a new car with the default parameters
!donkey createcar

The default parameters are:
- **`template=donkey2.py`** - Use `donkey2.py` template
- **`path=~/d2`** - Install to the `d2` directory in `$HOME`
- **`config=config_defaults.py`** - Copy the `config_defaults.py` file to `~/d2/config.py` to be used as default config file.

You can have a look at the `donkey2.py` template file just to get a feeling for how the car is set up:

In [ ]:
!cat donkey/donkeycar/templates/donkey2.py

## Tub related commands

Instead, let's have a look at the commands related to *Tubs*:
- `tubcheck`
- `tubclean`
- `tubhist`
- `tubplot`

### tubcheck

Check the *Tub* for invalid records. If `--fix` is `True`, deletes the invalid records.

In [ ]:
!donkey tubcheck ~/SageMaker/data/tub_8_18-02-09

### tubhist

Produce a histogram of record type frequency in the given tub

In [ ]:
!donkey tubhist ~/SageMaker/data/tub_8_18-02-09

Note that the histogram cannot be shown in the notebook since the expression that shows the figure, `matplotlib.pyplot.show()`, is run in a subshell. It will simply print the memory location of the figure.

However, we could try to run the python code in the notebook:

In [ ]:
%matplotlib inline
from donkeycar.management.base import ShowHistogram

hst = ShowHistogram()
hst.show_histogram('./data/tub_8_18-02-09', None)

### tubclean

`tubclean` is a tool for manually examining the *Tub*. It will start a webserver that you can connect to to browse images and metrics.

In [ ]:
!donkey tubclean ~/SageMaker/data/tub_8_18-02-09

The webserver runs on http://localhost:8886 per default.

**Sadly**, there doesn't seem to be a way to access that port from outside the notebook, hence you'll have to run this on your local host to get it to work.

### tubplot

Plot model predictions for angle and throttle against data from tubs.

**Sadly**, as of 2018-03-21, this command is not implemented correctly.

In [ ]:
!donkey tubplot ~/SageMaker/data/tub_8_18-02-09

## Video

There is a handly utility for creating a video from the images in the tub.

It requires the `config.py` file used by the car at the time of the recording, which we don't have. But we could try to use the default config file in `donkey/donkeycar/templates/config_defaults.py` instead:

In [ ]:
!donkey makemovie --tub ~/SageMaker/data/tub_8_18-02-09 --config donkey/donkeycar/templates/config_defaults.py

In [ ]:
ls -la

[Run video](../../tub_movie.mp4)

## Simulator

Finally, there is a Donkey car simulator that allows you to both record new data, and run trained models.

![Donkey Simulator](http://www.donkeycar.com/uploads/7/8/1/7/7817903/animation_orig.gif)

- http://docs.donkeycar.com/guide/simulator/

**As with `tubclean`**, the simulator runs as a webserver on your localhost. Thus, there is currently no way of accessing it on a *SageMaker Notebook*.

## Next

[Creating a Donkey Vehicle](./donkey-nn-vehicle.ipynb)